Here's how I load the data and reduce the memory usage of each dataframe.  I can save from 60% to 75% of memory usage on each dataframe.  
This method is inspired from this [kernel](https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65). I don't handle NANs at this point.  
Hope it helps.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
print('-' * 80)
print('train')
train = import_data('../input/application_train.csv')

print('-' * 80)
print('test')
test = import_data('../input/application_test.csv')

print('-' * 80)
print('bureau_balance')
bureau_balance = import_data('../input/bureau_balance.csv')

print('-' * 80)
print('bureau')
bureau = import_data('../input/bureau.csv')

print('-' * 80)
print('credit_card_balance')
credit_card = import_data('../input/credit_card_balance.csv')

print('-' * 80)
print('installments_payments')
installments = import_data('../input/installments_payments.csv')

print('-' * 80)
print('pos_cash_balance')
pos_cash = import_data('../input/POS_CASH_balance.csv')

print('-' * 80)
print('previous_application')
previous_app = import_data('../input/previous_application.csv')